In [ ]:
%matplotlib inline

import functions as fun
from collections import Counter
import ast
import pickle
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence



from scipy.sparse import csr_matrix

import pydot
import os
import numpy as np
import scipy as sp
import datetime
import matplotlib.pyplot as plt
from scipy.integrate import odeint 
import pandas as pd
import scipy as sp 
import string
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence
import sklearn.model_selection as ms
from tensorflow import keras
import tensorflow as tf
import tensorboard as ts
from keras.preprocessing.text import Tokenizer
#tf.keras.backend.clear_session()





## Training Data

In [22]:
h = ['claimID', 'claim', 'label', 'claimURL', 'reason', 'categories', 'speaker', 'checker', 'tags', 'articleTitle', 'publishDate', 'claimDate', 'entities']
df_train = pd.read_csv('../data/train.tsv',sep='\t',names=h,header=None)

df_train.head()

,claimID,claim,label,claimURL,reason,categories,speaker,checker,tags,articleTitle,publishDate,claimDate,entities
0,pomt-03627,Six out of 10 of the highest unemployment rate...,half-true,/ohio/statements/2013/may/06/chris-redfern/ohi...,When a couple of Statehouse Republicans prepar...,None,Chris Redfern,None,None,None,2013-05-06T06:00:00,2013-04-30,['None']
1,pomt-09611,No Democratic campaign for (Fla.) governor has...,true,/florida/statements/2010/jan/15/alex-sink/flor...,Florida's leading Republican candidate for gov...,None,Alex Sink,None,None,None,2010-01-15T13:59:00,2010-01-06,['None']
2,tron-00214,Forward an email for Jasmine,fiction!,https://www.truthorfiction.com/jasmine/,None,9-11-attack,None,None,None,Forward an email for Jasmine,"Mar 17, 2015",None,['None']
3,snes-04484,Pope Francis endorsed Donald Trump for president.,false,https://www.snopes.com/fact-check/pope-francis...,None,Junk News,None,Dan Evon,None,"Pope Francis Shocks World, Endorses Donald Tru...",10 July 2016,None,['None']
4,pomt-06704,Says Ron Paul insisted FEMA should be shut down.,true,/texas/statements/2011/sep/03/maureen-dowd/mau...,Commenting on the federal response to Hurrican...,None,Maureen Dowd,None,None,None,2011-09-03T06:00:00,2011-08-30,['None']


In [56]:
#df_train.reset_index(drop=False, inplace=True)
indexw = df_train.index[(df_train['label']=='true') | (df_train['label']=='false')].to_numpy()

indexw

array([    1,     3,     4, ..., 23997, 23999, 24002], dtype=int64)

## Snippet

In [16]:
h = ['snip_id', 'snip_1', 'snip_2','snip_url']
df_snip =  pd.DataFrame(columns=h)


dir = '..\data\snippets'
for filename in os.listdir(dir):
    
    df_row = pd.read_csv(os.path.join(dir, filename),names=h,sep='\t',header=None,engine='python',encoding='utf8' ,quoting=3)
    df_row.insert(loc = 0, column = 'claimID', value = filename )
    
    df_snip = df_snip.append(df_row)

df_snip.set_index(['claimID','snip_id'],inplace = False).head()    

snip_1  \
claimID    snip_id                                                      
abbc-00001 2        Record numbers march down Brisbane city to pro...   
           3        Fact check: Will Queensland's proposed abortio...   
           4        Queensland Council for Civil Liberties - Wikip...   
           5        Abortion law change expected from Queensland L...   
           6                                             The Struggle   

                                                               snip_2  \
claimID    snip_id                                                      
abbc-00001 2        Sep 4, 2018 ... THOUSANDS of people joined the...   
           3        Oct 3, 2018 ... Protestors have marched throug...   
           4        The Queensland Council for Civil Liberties (QC...   
           5        Jun 28, 2018 ... The Queensland Law Reform Com...   
           6        enforcing the State‟s controversial anti-march...   

                                                             snip_url  
claimID    snip_id                                                     
abbc-00001 2        http://catholicleader.com.au/news/record-numbe...  
           3        http://abortion-news.info/fact-check-will-quee...  
           4        https://en.wikipedia.org/wiki/Queensland_Counc...  
           5        http://www.abc.net.au/news/2018-06-29/abortion...  
           6        https://www.childrenbychoice.org.au/images/dow...

## Save training df to pickle.  (Remove rows with missing snippets)

In [24]:


df_snip = df_snip.loc[df_snip['claimID'].isin(df_train.claimID)]
df_train = df_train.loc[df_train['claimID'].isin(df_snip['claimID'].unique())]


print('train shape:',np.shape(df_train))
print('Snip shape:',np.shape(df_snip),'Unique:',len(df_snip['claimID'].unique()))

df_train.set_index('claimID',inplace = True)
df_snip.set_index(['claimID','snip_id'],inplace = True)


df_snip.to_pickle("./df_snip.pkl")
df_train.to_pickle("./df_train.pkl")

train shape: (24004, 13)
Snip shape: (219021, 5) Unique: 24004


In [31]:
df_snip = pd.read_pickle("./df_snip.pkl")
#df_train = pd.read_pickle("./df_train.pkl")

#df_train.reset_index(drop=False, inplace=True)
#df_snip.reset_index(drop=False, inplace=True)


In [25]:
y_train_num =  df_train['label'].astype("category").cat.codes.values

with open('y_train_num','wb') as i:
       pickle.dump(y_train_num, i) 

## Metadata set

In [27]:
df_meta = df_train[['claimURL','speaker','checker']].copy() 


#df_meta['articleTitle'] = df_meta['articleTitle'].str.lower()

df_meta['claimURL'] = df_meta.apply(lambda x: fun.clean_url(x.claimURL),axis=1)

df_meta['claimURL'] = df_meta['claimURL'].astype("category").cat.codes.values

df_meta['speaker'] = df_meta['speaker'].astype("category").cat.codes.values
df_meta['checker'] = df_meta['checker'].astype("category").cat.codes.values

    
df_meta.head() 

X_metadata = df_meta.to_numpy()

bin_enteties = fun.create_binary(df_train['entities'],30)

tags = df_train.apply(lambda x: str(x.tags).replace('[','').replace(']','').replace("'",'').split(',')   ,axis=1)

bin_tags = fun.create_binary(tags,30)



X_metadata = np.append(X_metadata, bin_enteties, axis=1)

with open('X_meta','wb') as i:
       pickle.dump(X_metadata, i)


df_meta_num = pd.DataFrame(data = np.hstack((df_train.index.to_numpy().reshape(-1,1),X_metadata))     )        
        
with open('df_meta_num','wb') as i:
       pickle.dump(df_meta_num, i)        



## Claim set

In [28]:
df_claim = df_train['claim'].str.lower().copy()

words_list = [i if type(i)==str else '' for i in df_claim]

t = Tokenizer()
t.fit_on_texts(words_list)

sequences = t.texts_to_sequences(words_list)

X_claim = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=50)


with open('X_claim','wb') as i:
       pickle.dump(X_claim, i)
            

            
df_claim_num = pd.DataFrame(data = np.hstack((df_train.index.to_numpy().reshape(-1,1),X_claim))   )        
        
with open('df_claim_num','wb') as i:
       pickle.dump(df_claim_num, i) 

## Snippet set

In [32]:
df_snip['snip_url'] = df_snip.apply(lambda x: fun.clean_url(x.snip_url),axis=1)

df_snip['snip_1'] = df_snip['snip_1'].str.lower()
df_snip['snip_2'] = df_snip['snip_2'].str.lower()

df_snip['snip_url'] = df_snip['snip_url'].astype("category").cat.codes.values


df_snip.reset_index(drop=False, inplace=True)


In [33]:

words_list1 = [i if type(i)==str else '' for i in df_snip['snip_1']]
sequences1 = t.texts_to_sequences(words_list1)
snip1 = tf.keras.preprocessing.sequence.pad_sequences(sequences1, maxlen=20)

words_list2 = [i if type(i)==str else '' for i in df_snip['snip_2']]
sequences2 = t.texts_to_sequences(words_list2)
snip2 = tf.keras.preprocessing.sequence.pad_sequences(sequences2, maxlen=80)


In [34]:


snip_info = df_snip[['claimID','snip_id','snip_url']].to_numpy()
snip = np.append(snip_info, np.append(snip1,snip2,axis=1), axis=1)


df_snip_num =pd.DataFrame(data=snip)
df_snip_num

with open('df_snip_num','wb') as i:
       pickle.dump(df_snip_num, i) 
        
        

In [35]:
df_snip_num.head()

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,100,101,102
0,abbc-00002,2,20830,0,0,0,0,0,0,0,...,5,1,776,100,8,93,16,186,5,452
1,abbc-00002,3,175,0,0,0,0,0,0,0,...,59,580,1,3849,681,5,2501,2372,12,13286
2,abbc-00002,4,7987,0,0,0,0,0,0,0,...,69,5,448,624,17744,12,1724,1,256,493
3,abbc-00002,5,8442,0,0,0,0,0,0,0,...,5,349,626,136,34,166,6,589,10,1188
4,abbc-00002,6,21017,0,0,0,0,0,0,0,...,1,165,5,6907,7814,714,1,1953,5018,2


## Flat Snippet set (very very slow)

In [234]:

snippet_list = []

for q,i in enumerate(df_train.index):

    row = []
    for i,r in df_snip_num[ df_snip_num[0]==i].iterrows():
       
        row.append(r[1:].to_list())
    row = [snip for snippets in row for snip in snippets]
    
    snippet_list.append(row)
    
   

KeyboardInterrupt: 

In [38]:
X_snip_flat =  tf.keras.preprocessing.sequence.pad_sequences(snippet_list, maxlen=1005)

with open('X_snip_flat','wb') as i:
       pickle.dump(X_snip_flat, i) 
        

## jj

In [ ]:
X_claim

In [39]:
print('claim',np.shape(X_claim))
print('meta',np.shape(X_metadata))
print('snip flat',np.shape(X_snip_flat))
print('y',np.shape(y_train_num))

claim (24004, 50)
meta (24004, 33)
snip flat (24004, 1005)
y (24004,)


array([[   0,  511,  127, ...,    0,    0,    0],
       [   0,   46,  127, ...,    0,    0,    0],
       [  35, 2243,  127, ...,    0,    0,    0],
       ...,
       [   0, 2181,  127, ...,    0,    0,    0],
       [   0, 1790,  127, ...,    0,    0,    0],
       [   0,  289,  127, ...,    0,    0,    0]], dtype=int16)

In [64]:
y_train_num[indexw]

array([96, 43, 96, ..., 43, 43, 43], dtype=int8)

In [62]:
df_train.loc[indexw]

,level_0,index,claimID,claim,label,claimURL,reason,categories,speaker,checker,tags,articleTitle,publishDate,claimDate,entities
1,1,1,pomt-09611,No Democratic campaign for (Fla.) governor has...,true,/florida/statements/2010/jan/15/alex-sink/flor...,Florida's leading Republican candidate for gov...,None,Alex Sink,None,None,None,2010-01-15T13:59:00,2010-01-06,['None']
3,3,3,snes-04484,Pope Francis endorsed Donald Trump for president.,false,https://www.snopes.com/fact-check/pope-francis...,None,Junk News,None,Dan Evon,None,"Pope Francis Shocks World, Endorses Donald Tru...",10 July 2016,None,['None']
4,4,4,pomt-06704,Says Ron Paul insisted FEMA should be shut down.,true,/texas/statements/2011/sep/03/maureen-dowd/mau...,Commenting on the federal response to Hurrican...,None,Maureen Dowd,None,None,None,2011-09-03T06:00:00,2011-08-30,['None']
8,8,8,pomt-00986,The 2012 National Survey on Drug Use and Healt...,true,/rhode-island/statements/2015/feb/09/susan-sha...,Vigorous debate surrounds efforts to legalize ...,None,Susan Shapiro,None,None,None,2015-02-09T00:01:00,2015-01-15,['United_States']
24,24,24,pomt-11794,"Says Doug Jones ""is for full-term abortion.""",false,/truth-o-meter/statements/2017/nov/21/kayla-mo...,As Alabama Republican Senate nominee Roy Moore...,None,Kayla Moore,None,None,None,2017-11-21T13:53:21,2017-11-17,['None']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23993,23993,23993,pomt-07162,"The average person ""will pay $6,000 more a yea...",true,/ohio/statements/2011/jun/13/tim-ryan/rep-tim-...,"Rep. Tim Ryan, a Democrat from Niles, Ohio, sh...",None,Tim Ryan,None,None,None,2011-06-13T12:35:28,2011-05-24,"['Paul_Ryan', 'Medicare_(United_States)']"
23994,23994,23994,snes-03990,Jennifer Aniston went on an angry tirade on th...,false,https://www.snopes.com/fact-check/jennifer-ani...,None,Junk News,None,Dan Evon,None,Jennifer Aniston Delivers Angry Response to ‘B...,20 September 2016,None,"['Brad_Pitt', 'Jennifer_Aniston', 'Angelina_Jo..."
23997,23997,23997,para-00050,When Abbott was Health Minister there was a sh...,false,http://pandora.nla.gov.au//pan/140601/20131209...,None,"['Health', 'Workforce Planning']",Tanya Plibersek,"Flynn Murphy, Peter Fray",None,Did Tony Abbott create a GP shortage?,"Tuesday, August 27, 2013 at 10:27 a.m.",None,['General_practitioner']
23999,23999,23999,snes-00146,Did Google Fail to Promote President Trump’s S...,false,https://www.snopes.com/fact-check/google-sotu-...,None,Politics,None,Dan Evon,None,Did Google Fail to Promote President Trump’s S...,30 August 2018,None,['None']


## Testing model

In [95]:
X_metadata[:,:3]

array([[   0,  511,  127],
       [   0,   46,  127],
       [  35, 2243,  127],
       ...,
       [   0, 2181,  127],
       [   0, 1790,  127],
       [   0,  289,  127]], dtype=int16)

In [91]:
type(y_train_num)

numpy.ndarray

In [99]:
X_train, X_test, y_train, y_test = ms.train_test_split(X_metadata, y_train_num, train_size=0.65,test_size=0.35, random_state=101)

In [134]:


embedding_size = 128

model = tf.keras.Sequential()

model.add(tf.keras.layers.Embedding(50000, embedding_size, input_shape= (X_train.shape[1],)))


model.add(tf.keras.layers.LSTM(units=128,  activation='sigmoid'))

model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

print(model.summary())

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=4, batch_size = 128,)

loss, accuracy = model.evaluate(X_test, y_test)

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 33, 128)           6400000   
_________________________________________________________________
lstm_17 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 129       
Total params: 6,531,713
Trainable params: 6,531,713
Non-trainable params: 0
_________________________________________________________________
None


ValueError: Input arrays should have the same number of samples as target arrays. Found 15602 input samples and 4373 target samples.

In [121]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

y_train

25000 train sequences
25000 test sequences


array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

In [123]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

np.shape(X_claim)

(24004, 50)

In [225]:
xx = X_metadata[indexw]
yy = y_train_num[indexw]

In [226]:
yy

yy= np.where(yy == 96, 1, 0)

yy

array([1, 0, 1, ..., 0, 0, 0])

In [227]:
x_train, x_test, y_train, y_test = ms.train_test_split(xx[50:], yy[50:], train_size=0.65,test_size=0.35, random_state=101)

In [230]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

# set parameters:
max_features = 5000
maxlen = 33
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 500
epochs = 3

print('Loading data...')


print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Loading data...
Pad sequences (samples x time)
x_train shape: (4340, 33)
x_test shape: (2338, 33)
Build model...


C:\Users\einar\anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4340 samples, validate on 2338 samples
Epoch 1/3
4340/4340 [==============================] - 1s 221us/step - loss: 0.5543 - accuracy: 0.7101 - val_loss: 0.5225 - val_accuracy: 0.7173
Epoch 2/3
4340/4340 [==============================] - 0s 110us/step - loss: 0.5064 - accuracy: 0.7459 - val_loss: 0.5108 - val_accuracy: 0.7335
Epoch 3/3
4340/4340 [==============================] - 0s 110us/step - loss: 0.4332 - accuracy: 0.8318 - val_loss: 0.5429 - val_accuracy: 0.7340


In [231]:

p = model.predict_classes(xx[:50])

In [232]:
r= yy[:50]

In [233]:
alle = len(r)
rett =0
for i,e in zip(p,r):
    if i==e:
        rett+=1
        
    
    
rett/alle   

0.78